Filtering Signals with overlapping window using MNE Notch Filter

In [ ]:
from mne.filter import notch_filter

def notch_filter_moving_window(data, movingwin,Fs,freqs, tau = 10, mt_bandwidth = 3):

    data = data.T
    N, C  = data.shape 
    Nwin = round(Fs * movingwin[0])  # Number of samples in window
    Nstep = round(movingwin[1] * Fs)  # Number of samples to step through
    Noverlap = Nwin - Nstep  # Number of points in overlap
    p = 0.05/Nwin

    x = np.arange(1, Noverlap + 1)
    smooth = 1. / (1 + np.exp(-tau * (x - Noverlap / 2) / Noverlap))  # Sigmoidal function
    # smooth = np.tile(smooth, (C, 1)).T  # Replicate for each channel

    winstart = np.arange(0, N - Nwin, Nstep)
    nw = len(winstart)  # Number of windows
    datafit = np.zeros((winstart[nw-1]+Nwin, C))


    for chIdx in range(0, C):
        for n in range(0, nw):
            indx = slice(winstart[n], winstart[n] + Nwin)
            datawin = data[indx, chIdx]

            # Apply notch filter to the window
            datafitwin = mne.filter.notch_filter(x = datawin, Fs = Fs, freqs = freqs, 
                                                method='spectrum_fit', mt_bandwidth=mt_bandwidth, 
                                                p_value = p, verbose='WARNING')
            if n==0:
                datafitwin0 = datafitwin.copy()    
            if n > 0:
                # Apply smoothing effect to the overlapping region
                datafitwinTemp = datafitwin.copy()
                datafitwin[:Noverlap] = smooth * datafitwin[:Noverlap] + (1 - smooth) * datafitwin0[Nwin - Noverlap:Nwin]
                datafitwin0 = datafitwinTemp.copy()

            datafit[indx, chIdx] = datafitwin

    return datafit